In [ ]:
import requests
from bs4 import BeautifulSoup
import time

base_url = 'https://rapzh.com'
total = 0
output_file = 'artist_info.csv'
def get_artist_info(url, total=0):
    print('正在爬取', url, '已经爬取', total, '条数据')
    count = 0
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    artist_names = []
    artist_urls = []
    for artist_link in soup.select('.ArtistItem-container'):
        artist_name = artist_link.get_text(strip=True)
        artist_url = base_url + artist_link['href']
        count += 1
        
        artist_names.append(artist_name)
        artist_urls.append(artist_url)

    next_page_link = soup.select_one('.Pagination-next')
    if next_page_link:
        next_page_url = base_url + next_page_link['href']
        
        with open(output_file, 'a') as f:
            for i in range(len(artist_names)):
                f.write(artist_names[i] + ',' + artist_urls[i] + '\n')
        
        get_artist_info(next_page_url, total + count)
        time.sleep(2)
    return 

# 从首页开始爬取
start_url = 'https://rapzh.com/artists'
get_artist_info(start_url, total)

In [15]:
import pandas as pd
# def get_chinese_rapper(df):
    
df = pd.read_csv('artist_info_clean.csv', header=None, names=['name', 'url'])
df['name'] = df['name'].fillna('')

In [18]:

english_names = ['PG One','Jony J', 'Higher Brothers', 'PO8', '3Bangz', 'Tizzy T', 'GAI', 'C-BLOCK'] 

# Create a boolean Series that is True for rows where the 'name' column is in the english_names list
english_name_mask = df['name'].isin(english_names)

# Combine the mask for Chinese characters and the mask for English names using the bitwise OR operator |
mask = df['name'].str.contains('[\u4e00-\u9fff]', regex=True) | english_name_mask

# Apply the mask to the DataFrame
df_chinese = df[mask]

In [20]:
df_chinese.to_csv('chinese_rappers.csv', index=False)